In [4]:
from Designer import SST, show
from math import pi, sqrt

In [66]:
def check_1382(Shp,A,Ix,Sx,Rx,Zx,Iy,Sy,Ry,Zy,J,Cw,D,B,T,W,Dsg,
               E=200000.,G=77000.,Fy=345.,
               Lx=0.,Kx=1.,Ly=0.,Ky=1,Cf=0.,Mfx1=0.,Mfx2=0.,Mfy1=0.,Mfy2=0.,braced=True,display=False):
    """13.8.2 and 13.8.3"""
    phi = 0.9
    if display:
        print('Section Properties and Dimensions:')
        show('Dsg,Shp,A,D,B,T,W,Rx,Ry,*1E6,Ix,*1E3,Sx,Zx,*1E6,Iy,*1E3,Sy,Zy,J,*1E9,Cw,*,E,G,Fy',data=locals())
        print('\nMember Data:')
        show('Lx,Kx,Ly,Ky,Cf,Mfx1,Mfx2,Mfy1,Mfy2,braced',data=locals())
        
    IShaped = Shp in SST.pfx2shp(['W','WWF','S'])
    
    if not IShaped:
        raise ValueError("Only I-shaped members are currently implemented.")
          
    #### Section Class:
        
    Cy = A*Fy * 1E-3

    fclass = wclass = 4
    
    bt = B/(2*T)    
    btlim = [x/sqrt(Fy) for x in [145,170,200]]
    for i in range(1,4):
        if bt <= btlim[i-1]:
            fclass = i
            break
            
    hw = (D-2*T)/W
    hwlim = [(x/sqrt(Fy))*(1-y*Cf/(phi*Cy)) for x,y in [(1100,.39),(1700,.61),(1900,.65)]]
    for i in range(1,4):
        if hw <= hwlim[i-1]:
            wclass = i
            break
    sclass = max(fclass,wclass)
    
    if display:
        print('\nSection Class:')
        show('b/2t=bt,class 1=btlim[0],class 2=btlim[1],class 3=btlim[2],flange class=fclass',data=locals())
        show('h/w=hw,class 1=hwlim[0],class 2=hwlim[1],class 3=hwlim[2],web class=wclass,sect. class=sclass',data=locals())

    if sclass > 3:
        if display:
            raise ValueError("'{}' is class 4. Strength checks not implemented.".format(Dsg))
        return False
    
    use_13_8_2 = IShaped and sclass <= 2
    
    # Moments and omega1 (13.8.5)
    
    Mfx = max(abs(Mfx1),abs(Mfx2))
    if Mfx:
        kappax = min(abs(Mfx1),abs(Mfx2))/Mfx
        if Mfx1*Mfx2 < 0:   # single curvature
            kappax = -kappax
        omega1x = max(0.4,0.6 - 0.4*kappax)
    else:
        kappax = 0.
        omega1x = 0.6
    Mfy = max(abs(Mfy1),abs(Mfy2))
    if Mfy:
        kappay = min(abs(Mfy1),abs(Mfy2))/Mfy
        if Mfy1*Mfy2 < 0:   # single curvature
            kappay = -kappay
        omega1y = max(0.4,0.6 - 0.4*kappay)
    else:
        kappay = 0.
        omega1y = 0.6
    if display:
        print('\n13.8.5: Values of omega1:')
        show('Mfx,kappax,omega1x,Mfy,kappay,omega1y',data=locals())
        
    # U1 (13.8.4)
    
    Cex = pi*pi*E*Ix/(Lx*Lx) * 1E-3
    Cey = pi*pi*E*Iy/(Ly*Ly) * 1E-3
    U1x = omega1x/(1-Cf/Cex)
    U1y = omega1y/(1-Cf/Cey)
    if display:
        print('\n13.8.4: Values of U1:')
        show('Cex,U1x,Cey,U1y',data=locals())
        
    # Cross-section strength (13.8.2 a) and 13.8.3 a))
    
    if braced:
        Cr = phi*A*Fy * 1E-3
        Mrx = phi*(Zx if sclass <= 2 else Sx)*Fy * 1E-6
        Mry = phi*(Zy if sclass <= 2 else Sy)*Fy * 1E-6
        sU1x,sU1y = U1x,U1y
        U1x,U1y = max(1.0,U1x),max(1.0,U1y)
        if use_13_8_2:
            intval = (Cf/Cr) + 0.85*U1x*Mfx/Mrx + 0.6*U1y*Mfy/Mry
            s = 'Cf/Cr,0.85*U1x*Mfx/Mrx,0.6*U1y*Mfy/Mry'
        else:
            intval = (Cf/Cr) + U1x*Mfx/Mrx + U1y*Mfy/Mry
            s = 'Cf/Cr,U1x*Mfx/Mrx,U1y*Mfy/Mry'
        if display:
            print('\n13.8.2 a), 13.8.3 a: Cross-sectional strength:')
            show('Cr,U1x,Mrx,U1y,Mry,'+s+',interaction value=intval',data=locals())
        U1x,U1y = sU1x,sU1y
        if intval > 1.0 and not display:
            return False
        
    # Overall member strength (13.8.2 b) and 13.8.3 b))
    
    klr = max(1*Lx/Rx,1*Ly/Ry)
    if Mfy == 0.:
        klr = 1*Lx/Rx   # uniaxial moment about x
    elif Mfx == 0.:
        klr = 1*Ly/Ry   # uniaxial moment about y
    Fe = pi*pi*E/(klr**2)
    lamda = sqrt(Fy/Fe)
    n = 1.34
    Cr = phi*A*Fy/(1+lamda**(2*n))**(1./n) * 1E-3
    
    klry = Ky*Ly/Ry
    Fey = pi*pi*E/(klry**2)
    lamday = sqrt(Fy/Fe)
    beta = min(0.6 + 0.5*lamday,0.85)
    
    sU1x,sU1y = U1x,U1y
    if not braced:
        U1x = U1y = 1.0
        
    Mrx = phi*(Zx if sclass <= 2 else Sx)*Fy * 1E-6
    Mry = phi*(Zy if sclass <= 2 else Sy)*Fy * 1E-6
    
    if use_13_8_2:
        intval = (Cf/Cr) + 0.85*U1x*Mfx/Mrx + beta*U1y*Mfy/Mry
        s = 'Cf/Cr,0.85*U1x*Mfx/Mrx,beta*U1y*Mfy/Mry'
    else:
        intval = (Cf/Cr) + U1x*Mfx/Mrx + U1y*Mfy/Mry
        s = 'Cf/Cr,U1x*Mfx/Mrx,U1y*Mfy/Mry'
        
    if display:
        print('\n13.8.2 b), 13.8.3 b): Overall Member Strength')
        show('kl/r=klr,Fe,lambda=lamda,Cr,KLy/ry=klry,lambda y=lamday,U1x,Mrx,beta,U1y,Mry,'+s+',interaction value=intval',
             data=locals())
    U1x,U1y = sU1x,sU1y
    if intval > 1.0 and not display:
        return False

In [67]:
SST.get('W250x33').call(check_1382,Cf=0.9*4170*.345*.5,
                        Lx=2000,Ly=2000,Mfx1=-200,Mfx2=150,Mfy1=50,Mfy2=20,display=1)

Section Properties and Dimensions:
Dsg = W250x33    
Shp = 1          
A   = 4170       
D   = 258        
B   = 146        
T   = 9.1        
W   = 6.1        
Rx  = 108        
Ry  = 33.7       
Ix  = 48.9 * 1E6 
Sx  = 379 * 1E3  
Zx  = 424 * 1E3  
Iy  = 4.73 * 1E6 
Sy  = 64.7 * 1E3 
Zy  = 99.5 * 1E3 
J   = 98.5 * 1E3 
Cw  = 73.2 * 1E9 
E   = 200000     
G   = 77000      
Fy  = 345        

Member Data:
Lx     = 2000  
Kx     = 1     
Ly     = 2000  
Ky     = 1     
Cf     = 647.4 
Mfx1   = -200  
Mfx2   = 150   
Mfy1   = 50    
Mfy2   = 20    
braced = True  

Section Class:
b/2t         = 8.022 
class 1      = 7.807 
class 2      = 9.152 
class 3      = 10.77 
flange class = 2     
h/w         = 39.31 
class 1     = 47.67 
class 2     = 63.61 
class 3     = 69.05 
web class   = 1     
sect. class = 2     

13.8.5: Values of omega1:
Mfx     = 200   
kappax  = -0.75 
omega1x = 0.9   
Mfy     = 50    
kappay  = 0.4   
omega1y = 0.44  

13.8.4: Values of U1:
Cex = 24130  
U1x = 0.9248 

In [68]:
SST.shps()

[(1, W, Wide-flange shapes),
 (2, S, S-shapes),
 (3, M, M-shapes),
 (4, HP, HP-shapes),
 (5, WWF, Welded wide-flange shapes),
 (6, C, Channels),
 (7, MC, Miscellaneous channels),
 (8, L, Single angles),
 (9, WT, T-shapes cut from W-shapes),
 (10, WWT, T-shapes cut from WWF-shapes),
 (11, 2L, Double angles - Short legs back-to-back),
 (12, 2L, Double angles - Long legs back-to-back),
 (13, 2L, Double angles - Equal legs back-to-back),
 (14, WRF, Welded reduced-flange shapes),
 (15, HS, Hollow structural sections - Square - CSA G40.20/21),
 (16, HS, Hollow structural sections - Rectangular - CSA G40.20/21),
 (17, HS, Hollow structural sections - Round - CSA G40.20/21),
 (18, SLB, Super light beams),
 (19, HA, Hollow structural sections - Square - ASTM A500),
 (20, HA, Hollow structural sections - Rectangular - ASTM A500),
 (21, HA, Hollow structural sections - Round - ASTM A500)]

In [69]:
SST.pfx2shp(['W','HS'])

[16, 1, 17, 15]

In [70]:
SST.get('W250x67').call(check_1382,Cf=400,
                        Lx=4000,Ly=4000,Mfx1=100,Mfx2=120,Mfy1=40,Mfy2=-50,braced=False,display=1)

Section Properties and Dimensions:
Dsg = W250x67    
Shp = 1          
A   = 8550       
D   = 257        
B   = 204        
T   = 15.7       
W   = 8.9        
Rx  = 110        
Ry  = 51         
Ix  = 104 * 1E6  
Sx  = 806 * 1E3  
Zx  = 901 * 1E3  
Iy  = 22.2 * 1E6 
Sy  = 218 * 1E3  
Zy  = 332 * 1E3  
J   = 625 * 1E3  
Cw  = 324 * 1E9  
E   = 200000     
G   = 77000      
Fy  = 345        

Member Data:
Lx     = 4000  
Kx     = 1     
Ly     = 4000  
Ky     = 1     
Cf     = 400   
Mfx1   = 100   
Mfx2   = 120   
Mfy1   = 40    
Mfy2   = -50   
braced = False 

Section Class:
b/2t         = 6.497 
class 1      = 7.807 
class 2      = 9.152 
class 3      = 10.77 
flange class = 1     
h/w         = 25.35 
class 1     = 55.74 
class 2     = 83.11 
class 3     = 92.27 
web class   = 1     
sect. class = 1     

13.8.5: Values of omega1:
Mfx     = 120    
kappax  = 0.8333 
omega1x = 0.4    
Mfy     = 50     
kappay  = -0.8   
omega1y = 0.92   

13.8.4: Values of U1:
Cex = 12830  
U1x = 0